In [5]:
import pandas as pd
import numpy as np
import joblib
import uvicorn
import pickle
from fastapi import FastAPI, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import random
from pydantic import BaseModel
import nest_asyncio
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from twilio.rest import Client
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime, timedelta
from pyngrok import ngrok
import threading

# Ngrok Authentication
NGROK_AUTH_TOKEN = "28WOZmFsePzDohg4URD5MU7tgBN_3onpraLYjUsXNXoB4iiro"
ALLOWED_ORIGINS = ["https://disease-surveillance-ai-new.vercel.app", "http://localhost:3000"]

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=ALLOWED_ORIGINS,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# -------------------------------
# Module 1 :- Disease Trend Analysis (Disease Case Forecasting)
# -------------------------------
with open("/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Disease Trend Analysis (Case Forecasting)/Disease Trend Analysis Model/trained_arima_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

class CaseInput(BaseModel):
    today_cases: int

@app.post("/disease_forecast/predict")
def get_forecast(data: CaseInput, background_tasks: BackgroundTasks):
    today_cases = data.today_cases
    days = 7
    forecast_values = trained_model.forecast(steps=days)
    forecast_values = [today_cases + (val - forecast_values[0]) for val in forecast_values]
    future_dates = [(datetime.today() + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, days + 1)]
    forecast_data = [{"date": date, "predicted_cases": round(value)} for date, value in zip(future_dates, forecast_values)]

    if any(value > 600 for value in forecast_values):
        sms_alert = "⚠️Alert: Our forecast indicates disease cases will exceed 600, signaling a potential pandemic risk. Please immediately review detailed dashboard data and implement emergency protocols. Timely action is critical to safeguard public health. Contact support for further guidance. Thank you. Immediate response required. Please stay safe, act now immediately without delay."
        email_alert = """Health Authority Team,

Our AI-Powered Predictive Analytics System for Disease Surveillance & Outbreak Management has generated an urgent alert. Our forecasting model predicts that the number of disease cases will exceed 600 in the coming days, which may signal an emerging pandemic risk.

Alert Details:

Forecast Threshold: Cases > 600

Implication: Potential pandemic risk requiring immediate review

Timestamp: [Insert Date & Time]

We strongly recommend that you review the detailed analytics on the dashboard and take prompt action to mobilize appropriate resources. Your timely response is critical to mitigating the risk and protecting public health.

Please do not hesitate to contact the project team for further insights or assistance.

Sincerely,
HealthGuard.AI"""
        whatsapp_alert = "⚠️Alert: Our forecast indicates that disease cases will exceed 600, signaling a potential pandemic risk. Please immediately review dashboard data and implement emergency protocols. Your prompt action is critical to safeguard public health. Contact our support team for further guidance. Thank you for your prompt attention. Immediate response is required."
        background_tasks.add_task(send_email_alert, "⚠️Urgent Alert: Disease Forecasting Predicting Possible Pandemic Incoming", email_alert)
        background_tasks.add_task(send_sms_alert, sms_alert)
        background_tasks.add_task(send_whatsapp_alert, whatsapp_alert)

    return {"forecast": forecast_data, "message": "Forecast generated successfully"}

# -------------------------------
# Module 2 :- Disease Outbreak Prediction
# -------------------------------
xgboost_model_filename = "/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Disease Outbreak Prediction/xgboost_model.pkl"
scaler_filename = "/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Disease Outbreak Prediction/scaler.pkl"
xgboost_model = joblib.load(xgboost_model_filename)
scaler = joblib.load(scaler_filename)
expected_features = scaler.feature_names_in_

class InputData(BaseModel):
    total_cases: int
    active_cases: int
    mild_cases: int
    moderate_cases: int
    severe_cases: int
    mortality_rate: float
    recovery_rate: float
    occupied_beds: int
    occupied_ventilators: int
    occupied_oxygen: int
    total_icu_beds: int
    icu_utilization: int
    emergency_admission_rate: int
    resource_load: int

def classify_outbreak_risk(prediction):
    historical_outbreak_values = np.random.randint(50, 5000, size=100)
    low_threshold = np.percentile(historical_outbreak_values, 25)
    moderate_threshold = np.percentile(historical_outbreak_values, 50)
    high_threshold = np.percentile(historical_outbreak_values, 75)
    if prediction < low_threshold:
        return "Low Risk"
    elif prediction < moderate_threshold:
        return "Moderate Risk"
    elif prediction < high_threshold:
        return "High Risk"
    else:
        return "Critical Risk"

@app.post("/disease_outbreak/predict")
def predict_outbreak(data: InputData, background_tasks: BackgroundTasks):
    try:
        new_data = pd.DataFrame([data.model_dump()])
        feature_mapping = {
            "total_cases": "Total Cases",
            "active_cases": "Active Cases",
            "mild_cases": "Mild Cases",
            "moderate_cases": "Moderate Cases",
            "severe_cases": "Severe Cases",
            "mortality_rate": "Mortality Rate",
            "recovery_rate": "Recovery Rate",
            "occupied_beds": "Occupied Beds",
            "occupied_ventilators": "Occupied Ventilators",
            "occupied_oxygen": "Occupied Oxygen",
            "total_icu_beds": "Total ICU Beds in Hospital",
            "icu_utilization": "ICU Utilization",
            "emergency_admission_rate": "Emergency Admission Rate",
            "resource_load": "Resource Load"
        }
        new_data.rename(columns=feature_mapping, inplace=True)
        new_data = new_data[expected_features]
        new_data_scaled = scaler.transform(new_data)
        xgb_prediction = xgboost_model.predict(new_data_scaled)[0]
        prediction_value = float(xgb_prediction)
        risk_category = classify_outbreak_risk(prediction_value)

        if risk_category in ["High Risk", "Critical Risk"]:
            sms_alert = f"⚠️Alert: Disease outbreak classified as {risk_category}. Immediate action required. Please review the dashboard and initiate emergency protocols. Contact support for guidance."
            whatsapp_alert = f"⚠️Urgent Alert: Our prediction model has classified the disease outbreak as {risk_category}. Immediate action is required. Kindly review the detailed analytics on the dashboard and activate your emergency response protocols. For further assistance, please contact our support team."
            email_alert = f"""Dear Health Authority Team,

Our AI-powered disease surveillance system has classified the current outbreak risk as {risk_category}. This elevated risk level requires immediate review and activation of emergency response protocols. Please log into the dashboard for detailed analytics and initiate appropriate contingency measures.

Your prompt attention to this critical situation is essential.

Sincerely,
HealthGuard.AI"""
            background_tasks.add_task(send_email_alert, "⚠️Urgent Alert: Elevated Disease Outbreak Risk Detected",email_alert)
            background_tasks.add_task(send_sms_alert, sms_alert)
            background_tasks.add_task(send_whatsapp_alert, whatsapp_alert)

        return {"prediction": prediction_value, "risk_category": risk_category, "message": "Prediction completed"}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# -------------------------------
# Module 3 :- Symptoms Based Disease Classification
# -------------------------------
model = joblib.load("/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Symptoms Based Disease Classification/disease_model.pkl")
label_encoder = joblib.load("/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Symptoms Based Disease Classification/label_encoder.pkl")
dataset_path = "/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Symptoms Based Disease Classification/Training.csv"
train_df = pd.read_csv(dataset_path)
train_df = train_df.drop(columns=["Unnamed: 133"], errors="ignore")
symptom_columns = list(train_df.columns[:-1])

class SymptomInput(BaseModel):
    symptoms: list

@app.post("/disease_classify/predict")
def predict_disease(input_data: SymptomInput):
    symptoms_vector = np.zeros((1, len(symptom_columns)))
    for symptom in input_data.symptoms:
        if symptom in symptom_columns:
            symptoms_vector[0][symptom_columns.index(symptom)] = 1
    prediction = model.predict(symptoms_vector)
    predicted_disease = label_encoder.inverse_transform(prediction)[0]
    return {"predicted_disease": predicted_disease}

# -------------------------------
# Module 5 :- Social Media And News API
# -------------------------------
def load_model(model_name="distilgpt2"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

districts = [
    "Ahmedabad", "Amreli", "Anand", "Aravalli", "Banaskantha", "Bharuch", "Bhavnagar", "Botad", "Chhota Udaipur", "Dahod",
    "Dang", "Devbhoomi Dwarka", "Gandhinagar", "Gir Somnath", "Jamnagar", "Junagadh", "Kheda", "Kutch", "Mahisagar", "Mehsana",
    "Morbi", "Narmada", "Navsari", "Panchmahal", "Patan", "Porbandar", "Rajkot", "Sabarkantha", "Surat", "Surendranagar",
    "Tapi", "Vadodara", "Valsad"
]
villages = [
    "Dholka", "Viramgam", "Maliya", "Dediapada", "Mandvi", "Dhrol", "Bagasara", "Halol", "Kalol", "Lathi", "Rapar", "Babra", "Una"
]
diseases = ["Dengue", "Malaria", "Chikungunya", "Swine Flu", "Typhoid", "Hepatitis", "COVID-19", "Zika Virus", "Tuberculosis", "Cholera"]

def generate_text(prompt, tokenizer, model, max_length=150):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text.strip().replace("\n", " ")

def generate_news():
    tokenizer, model = load_model()
    disease = random.choice(diseases)
    location = random.choice(districts + villages)
    news_templates = [
        f"🚨 Breaking News: {disease} outbreak reported in {location}. Health officials urge immediate action.",
        f"⚠️ Health Alert: {location} sees a sharp rise in {disease} cases. Hospitals are preparing for more patients.",
        f"📢 Government Notice: Authorities in {location} impose new health restrictions due to rising {disease} cases.",
        f"🦠 Medical Update: {location} records its highest {disease} cases this month. Doctors warn of potential complications.",
        f"📊 Statistics Report: {disease} cases in {location} have increased by {random.randint(1,100)}% in the last week. Citizens urged to stay cautious.",
        f"🔬 Research Findings: Scientists in {location} study {disease} spread patterns to prevent future outbreaks.",
        f"🏥 Healthcare Update: Hospitals in {location} report increased admissions due to {disease}. Medical resources are under strain.",
        f"🌍 Global Concern: {disease} cases in {location} spark concerns of possible regional outbreak. Experts recommend immediate measures.",
        f"📰 Local News: {location} witnesses unusual {disease} spike. Public urged to follow preventive guidelines strictly."
    ]
    prompt = random.choice(news_templates)
    return generate_text(prompt, tokenizer, model, max_length=150)

def generate_social_media_post():
    tokenizer, model = load_model()
    disease = random.choice(diseases)
    location = random.choice(districts)
    social_media_templates = [
        f"🚨 Stay Safe! {disease} cases are rising in {location}. Follow safety measures and avoid crowded places! #HealthAlert",
        f"⚠️ Attention {location} residents! {disease} infections are increasing. Wash your hands regularly and stay protected!",
        f"🦠 {disease} outbreak in {location}. Health officials recommend vaccination and hygiene measures to stop the spread.",
        f"📢 Urgent! {disease} cases in {location} have surged. Wear masks and maintain social distancing! #StaySafe",
        f"🔬 Researchers in {location} are working on a new treatment for {disease}. Stay tuned for more updates!",
        f"🏥 Hospitals in {location} are seeing an increase in {disease} cases. Please follow medical guidelines to protect yourself.",
        f"📊 The latest data shows a rise in {disease} cases in {location}. Be cautious and stay informed!",
        f"🌍 {location} is facing a growing {disease} crisis. Authorities urge people to take all necessary precautions.",
        f"📰 Local Update: {location} sees an increase in {disease} cases. Help stop the spread by following health guidelines!"
    ]
    prompt = random.choice(social_media_templates)
    return generate_text(prompt, tokenizer, model, max_length=150)

daily_news_count = 0
daily_social_count = 0
reset_time = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)

@app.get("/disease_news/predict")
def get_news(background_tasks: BackgroundTasks):
    global daily_news_count
    news_article = generate_news()
    daily_news_count += 1

    alert_message = None
    if daily_news_count >= 10:
        sms_alert = "⚠️Alert: Over 10 disease-related news articles detected today. Please check the dashboard for details and take prompt action."
        whatsapp_alert = """Dear Health Authority Team,

Our AI-powered surveillance system has detected a significant surge in disease-related news coverage, with over 10 articles published today. This uptick may indicate heightened public concern or emerging outbreak signals.

Action Required:

Please review the detailed analytics on the dashboard.

Evaluate the situation and consider any necessary response measures.

Your prompt attention to this matter is critical.

Sincerely,
HealthGuard.AI"""
        email_alert = "⚠️Urgent Alert: Our system has detected more than 10 disease-related news articles today. Kindly review the dashboard for detailed analytics and consider immediate response measures. Your prompt action is crucial."
        background_tasks.add_task(send_email_alert, "⚠️Urgent Alert: Surge in Disease-Related News Coverage Detected", email_alert)
        background_tasks.add_task(send_sms_alert, sms_alert)
        background_tasks.add_task(send_whatsapp_alert, whatsapp_alert)

    return {"news": news_article, "alert": sms_alert}

@app.get("/disease_social/predict")
def get_social_media_posts(background_tasks: BackgroundTasks):
    global daily_social_count
    social_post = generate_social_media_post()
    daily_social_count += 1

    alert_message = None
    if daily_social_count >= 10:
        sms_alert = "⚠️Alert: Over 10 disease-related social media posts detected today. Please check the dashboard for details and take prompt action."
        whatsapp_alert = """Dear Health Authority Team,

Our AI-powered surveillance system has detected a significant surge in disease-related social media activity, with over 10 posts published today. This increase may indicate rising public concern or early outbreak signals.

Action Required:

Please review the detailed analytics on the dashboard.

Evaluate the situation and consider initiating appropriate response measures.

Your prompt attention to this matter is essential.

Sincerely,
HealthGuard.AI"""
        email_alert = "⚠️Urgent Alert: Our system has detected more than 10 disease-related social media posts today. Kindly review the dashboard for detailed analytics and consider immediate response measures. Your prompt action is critical."
        background_tasks.add_task(send_email_alert, "⚠️Urgent Alert: Surge in Disease-Related Social Media Activity Detected", email_alert)
        background_tasks.add_task(send_sms_alert, sms_alert)
        background_tasks.add_task(send_whatsapp_alert, whatsapp_alert)

    return {"social_post": social_post, "alert": sms_alert}

# -------------------------------
# Module 6 :- Automated Alerts and Notifications (Configuration)
# -------------------------------
# Overriding alert configuration with your credentials
EMAIL_SENDER = "harshpatel080503@gmail.com"
EMAIL_PASSWORD = "fzji awzn kdwn rcxq"  # Use App Password if using Gmail
TWILIO_SID = "AC4bbe22d3201f5ecd0cec29d4df7d232b"
TWILIO_AUTH_TOKEN = "b6dc34b392146000797ae7e011bccea4"
TWILIO_PHONE_NUMBER = "+17856755684"
ALERT_RECEIVER_EMAIL = "parva300504@gmail.com"
ALERT_RECEIVER_PHONE = "+919510493577"
# For WhatsApp, numbers must be prefixed with 'whatsapp:'
TWILIO_WHATSAPP_NUMBER = "whatsapp:+17856755684"
ALERT_RECEIVER_WHATSAPP = "whatsapp:+916354394636"

def send_email_alert(subject, message):
    msg = MIMEMultipart()
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = ALERT_RECEIVER_EMAIL
    msg.attach(MIMEText(message, "plain"))
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(EMAIL_SENDER, EMAIL_PASSWORD)
            smtp.send_message(msg)
        print("✅ Email Sent Successfully!")
    except Exception as e:
        print(f"❌ Email Sending Failed: {e}")

def send_sms_alert(message):
    try:
        client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)
        client.messages.create(
            body=message,
            from_=TWILIO_PHONE_NUMBER,
            to=ALERT_RECEIVER_PHONE
        )
        print("✅ SMS Sent Successfully!")
    except Exception as e:
        print(f"❌ SMS Sending Failed: {e}")

def send_whatsapp_alert(message):
    try:
        client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)
        client.messages.create(
            body=message,
            from_=TWILIO_WHATSAPP_NUMBER,
            to=ALERT_RECEIVER_WHATSAPP
        )
        print("✅ WhatsApp Alert Sent Successfully!")
    except Exception as e:
        print(f"❌ WhatsApp Alert Sending Failed: {e}")

def reset_daily_counters():
    global daily_news_count, daily_social_count, reset_time
    while True:
        now = datetime.now()
        if now >= reset_time:
            daily_news_count = 0
            daily_social_count = 0
            reset_time = now.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)
            print("🔄 Daily counters reset!")
        threading.Event().wait(60)

reset_thread = threading.Thread(target=reset_daily_counters, daemon=True)
reset_thread.start()

@app.get("/alerts/check")
def check_alerts():
    return {
        "daily_news_count": daily_news_count,
        "daily_social_count": daily_social_count,
        "reset_time": reset_time.strftime("%Y-%m-%d %H:%M:%S"),
        "message": "Alerts are triggered when more than 10 news articles or social posts are detected in a day."
    }

if __name__ == "__main__":
    nest_asyncio.apply()
    public_url = ngrok.connect(8000).public_url
    print(f"FastAPI running on: {public_url}")
    uvicorn.run(app, host="0.0.0.0", port=8000)


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-29' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

FastAPI running on: https://2c09-35-230-174-211.ngrok-free.app


INFO:     Started server process [17605]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2409:40c1:9:1d3a:399b:3370:c9a3:10c8:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2409:40c1:9:1d3a:399b:3370:c9a3:10c8:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2409:40c1:9:1d3a:399b:3370:c9a3:10c8:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2409:40c1:9:1d3a:399b:3370:c9a3:10c8:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2409:40c1:9:1d3a:399b:3370:c9a3:10c8:0 - "POST /disease_outbreak/predict HTTP/1.1" 200 OK
✅ Email Sent Successfully!
✅ SMS Sent Successfully!
❌ WhatsApp Alert Sending Failed: HTTP 400 error: Unable to create record: Twilio could not find a Channel with the specified From address


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [17605]
